<a href="https://colab.research.google.com/github/d1m3j1/Machine-Learning/blob/main/Linear_Regression_with_SGD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# Example of a Train-Test Test Harness
from random import seed
from random import randrange
from csv import reader
from math import sqrt

In [14]:

# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset


In [15]:
# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())
  
  

In [16]:
# Find the min and max values for each column
def dataset_minmax(dataset):
    minmax = list()
    for i in range(len(dataset[0])):
        col_values = [row[i] for row in dataset]
        value_min = min(col_values)
        value_max = max(col_values)
        minmax.append([value_min, value_max])
    return minmax



In [17]:

# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])



In [18]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(int(n_folds)):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split



In [19]:
# Calculate root mean squared error
def rmse_metric(actual, predicted):
    sum_error = 0.0
    for i in range(len(actual)):
        prediction_error = predicted[i] - actual[i]
        sum_error += (prediction_error ** 2)
    mean_error = sum_error / float(len(actual))
    return sqrt(mean_error)



In [20]:
# Evaluate an algorithm using a train/test split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        rmse = rmse_metric(actual, predicted)
        scores.append(rmse)
    return scores




In [21]:
# Make a prediction with coefficients
def predict(row, coefficients):
    yhat = coefficients[0]
    for i in range(len(row)-1):
        yhat += coefficients[i + 1] * row[i]
    return yhat

        


In [22]:
# Estimate linear regression coefficients using stochastic gradient descent
def coefficients_sgd(train, l_rate, n_epoch):
    coef = [0.0 for i in range(len(train[0]))]
    for _ in range(n_epoch):
        for row in train:
            yhat = predict(row, coef)
            error = yhat - row[-1]
            coef[0] = coef[0] - l_rate * error
            for i in range(len(row)-1): 
                coef[i + 1] = coef[i + 1] - l_rate * error * row[i]
    print(coef)  
    return coef




In [23]:
# Linear Regression Algorithm With Stochastic Gradient Descent
def linear_regression_sgd(train, test, l_rate, n_epoch):
    predictions = list()
    coef = coefficients_sgd(train, l_rate, n_epoch)
    for row in test:
        yhat = predict(row, coef)
        predictions.append(yhat)
    return(predictions)


In [24]:
seed(1)
# load and prepare data
filename = 'wineQualityData.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0])):
    str_column_to_float(dataset, i)
# normalize
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)
# evaluate algorithm
n_folds = 10
l_rate = 0.001
n_epoch = 50
scores = evaluate_algorithm(dataset, linear_regression_sgd, n_folds, l_rate, n_epoch)
print('Scores: %s' % scores)
print('Mean RMSE: %.3f' % (sum(scores)/float(len(scores))))

[0.3291773261336548, 0.020839422049727937, -0.3085206302038366, 0.026240769698064888, 0.16019487149816633, -0.06333091455152685, 0.12168674890101895, -0.0671919257432176, -0.003469800274182276, 0.046117850319115965, 0.14382724989985654, 0.3794246755633594]
[0.32881299826746657, 0.011571454650751201, -0.311093709319755, 0.028151971636303532, 0.14887098900077145, -0.05938568004380806, 0.1302226726913123, -0.05718731371002893, -0.007390648146691016, 0.044657254565102236, 0.15070814413234718, 0.3798866247041621]
[0.3246762737357739, 0.015987569567351062, -0.31526289426507326, 0.03652626988344421, 0.15804397928773864, -0.05551485650196283, 0.12159826846902572, -0.05799258198616639, -0.004966200180362066, 0.05260805696152286, 0.13978438801257362, 0.38135359845095707]
[0.3258193984436971, 0.02454026481725514, -0.3108265168817948, 0.026241589182899712, 0.152874407516737, -0.0709954283874715, 0.1220294317308766, -0.05815560364473801, -0.009211454592793942, 0.0484981142768662, 0.1447562571680570